# Quenching the Thirst for Insights

*Water is one of the most valueable ressources not only to us humans but for many other living organisms as well. Although, water security often gets less media attention than climate change topics, water scarcity is a serious issue that affects us all. **Water scarcity** is created where the water withdrawal from a basin exceeds its recharge and **water stress level** is the freshwater withdrawal as a proportion of available freshwater resources. Managing water as a ressource can be challenging. Therefore, the goal of this challenge is to predict water levels to help Acea Group preserve precious waterbodies.
In this notebook we are getting an overview of the challenge first and then explore the various datasets and see if we can find some interesting insights.*

(Additional information on water scarcity and water stress level can be found [in this notebook](https://www.kaggle.com/iamleonie/impact-potential-analysis-of-water-use-efficiency).)

# Challenge Overview

Since Acea Groups is an Italian multiutility operator, we are looking at datasets containing information about **waterbodies in Italy**. 
The distribution of datasets for the type of waterbody is different. There are four waterbody types: aquifier, water spring, lake and river. The datasets are not equally distributed over the waterbody types, as shown below:
> This competition uses nine different datasets, completely independent and not linked to each other. 


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import re
import datetime

from datetime import datetime
import matplotlib.cm as cm
from geopy.geocoders import Nominatim
import folium
from tqdm import tqdm 

from math import radians, cos, sin, asin, sqrt
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import gc
import lightgbm as lgb



PATH = "../input/acea-water-prediction/"
aquifer_auser_df = pd.read_csv(f"{PATH}Aquifer_Auser.csv")
aquifer_doganella_df = pd.read_csv(f"{PATH}Aquifer_Doganella.csv")
aquifer_luco_df = pd.read_csv(f"{PATH}Aquifer_Luco.csv")
aquifer_petrignano_df = pd.read_csv(f"{PATH}Aquifer_Petrignano.csv")

lake_biliancino_df = pd.read_csv(f"{PATH}Lake_Bilancino.csv")

river_arno_df = pd.read_csv(f"{PATH}River_Arno.csv")

water_spring_amiata_df = pd.read_csv(f"{PATH}Water_Spring_Amiata.csv")
water_spring_lupa_df = pd.read_csv(f"{PATH}Water_Spring_Lupa.csv")
water_spring_madonna_df = pd.read_csv(f"{PATH}Water_Spring_Madonna_di_Canneto.csv")

datasets = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if '.csv' in filename:
            datasets += list([filename])

datasets_df = pd.DataFrame(columns=['filename'], data=datasets)
datasets_df['waterbody_type'] = datasets_df.filename.apply(lambda x: x.split('_')[0])

f, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,4))

sns.countplot(datasets_df.waterbody_type, palette=['lightblue'])
ax.set_ylabel('Number of Datasets', fontsize=14)
ax.set_xlabel('Waterbody Type', fontsize=14)
ax.set_xticklabels(labels=['Aquifier', 'Water Spring', 'Lake', 'River'], fontsize=14)
plt.show()

Since we are working with different datasets, we also have different columns.
> The reality is that each waterbody has such unique characteristics that their attributes are not linked to each other. 

Let's check which are column categories the datasets have in common. As shown below, the columns `Date`, `Rainfall` and `Temperature` appear in all datasets. The columns `Lake Level` and `Volume` are waterbody unique columns.

Additionally, for each waterbody a different feature has to be predicted. In below figure the target variable is marked with a lightblue box.

**The goal is to create four mathematical models, one for each category of waterbody (acquifers, water springs, river, lake) to predict the amount of water in each unique waterbody.**

In [ ]:
temp_df = pd.DataFrame({'column_name' : aquifer_auser_df.columns, 'waterbody_type':'Aquifier'})
temp_df = temp_df.append(pd.DataFrame({'column_name' : aquifer_doganella_df.columns, 'waterbody_type':'Aquifier'}))
temp_df = temp_df.append(pd.DataFrame({'column_name' : aquifer_luco_df.columns, 'waterbody_type':'Aquifier'}))
temp_df = temp_df.append(pd.DataFrame({'column_name' : aquifer_petrignano_df.columns, 'waterbody_type':'Aquifier'}))
temp_df = temp_df.append(pd.DataFrame({'column_name' : lake_biliancino_df.columns, 'waterbody_type':'Lake'}))
temp_df = temp_df.append(pd.DataFrame({'column_name' : river_arno_df.columns, 'waterbody_type':'River'}))
temp_df = temp_df.append(pd.DataFrame({'column_name' : water_spring_amiata_df.columns, 'waterbody_type':'Water Spring'}))
temp_df = temp_df.append(pd.DataFrame({'column_name' : water_spring_lupa_df.columns, 'waterbody_type':'Water Spring'}))
temp_df = temp_df.append(pd.DataFrame({'column_name' : water_spring_madonna_df.columns, 'waterbody_type':'Water Spring'}))

def get_column_category(x):
    if 'Date' in x:
        return 'Date'
    elif 'Rainfall' in x:
        return 'Rainfall'
    elif 'Depth' in x:
        return 'Depth to Groundwater'
    elif 'Temperature' in x:
        return 'Temperature'
    elif 'Volume' in x:
        return 'Volume'
    elif 'Hydrometry' in x:
        return 'Hydrometry'
    elif 'Lake_Level' in x:
        return 'Lake Level'
    elif 'Flow_Rate' in x:
        return 'Flow Rate'
    else:
        return x

temp_df['column_category'] = temp_df.column_name.apply(lambda x: get_column_category(x))

temp_df = temp_df.groupby('waterbody_type').column_category.value_counts().to_frame()
temp_df.columns = ['amount']
temp_df =temp_df.reset_index(drop=False)
temp_df = temp_df.pivot(index='waterbody_type', columns='column_category')['amount']
temp_df = temp_df.notna()

temp_df = temp_df[['Date','Rainfall', 'Temperature',  'Depth to Groundwater', 'Flow Rate', 'Hydrometry', 'Lake Level',
       'Volume']]

f, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,6))

sns.heatmap(temp_df, cmap='Blues', linewidth=1, ax=ax)

ax.set_ylabel('Waterbody Type', fontsize=16)
ax.set_xlabel('Column Category', fontsize=16)
ax.set_title('Heatmap of Column Categories in Dataset', fontsize=16)
ax.add_patch(Rectangle((3, 0), 1, 1, fill=True, alpha=1, color='dodgerblue', lw=0))
ax.add_patch(Rectangle((6, 1), 1, 1, fill=True, alpha=1, color='dodgerblue', lw=0))
ax.add_patch(Rectangle((4, 1), 1, 1, fill=True, alpha=1, color='dodgerblue', lw=0))
ax.add_patch(Rectangle((5, 2), 1, 1, fill=True, alpha=1, color='dodgerblue', lw=0))
ax.add_patch(Rectangle((4, 3), 1, 1, fill=True, alpha=1, color='dodgerblue', lw=0))
ax.annotate('Target \n Value', (3.1, 0.6), color='white', weight='bold', fontsize=14)
ax.annotate('Target \n Value', (6.1, 1.6), color='white', weight='bold', fontsize=14)
ax.annotate('Target \n Value', (4.1, 1.6), color='white', weight='bold', fontsize=14)
ax.annotate('Target \n Value', (5.1, 2.6), color='white', weight='bold', fontsize=14)
ax.annotate('Target \n Value', (4.1, 3.6), color='white', weight='bold', fontsize=14)

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(14) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(14)
    tick.label.set_rotation('horizontal')
plt.show()

# Preprocessing



In [ ]:
# Dropping Rows with Full NaN Values
def drop_nan_rows(df):
    columns = [c for c in df.columns if c != 'Date']
    df_mod = df[columns]
    df_mod.dropna(axis = 0, how = 'all', inplace = True)
    df_mod['Date'] = df['Date']
    print(f"Dropped {len(df)-len(df_mod)} rows")
    return df_mod

#aquifer_auser_df = drop_nan_rows(aquifer_auser_df)
aquifer_doganella_df = drop_nan_rows(aquifer_doganella_df)
#aquifer_luco_df = drop_nan_rows(aquifer_luco_df)
#aquifer_petrignano_df = drop_nan_rows(aquifer_petrignano_df)
#lake_biliancino_df = drop_nan_rows(lake_biliancino_df)
river_arno_df = drop_nan_rows(river_arno_df)
#water_spring_amiata_df = drop_nan_rows(water_spring_amiata_df)
#water_spring_lupa_df = drop_nan_rows(water_spring_lupa_df)
water_spring_madonna_df = drop_nan_rows(water_spring_madonna_df)

## Finding Duplicate Features
The datasets for Lake Biliancino and River Arno share the column names `Rainfall_S_Piero`, `Rainfall_Mangona`, `Rainfall_S_Agata`, `Rainfall_Cavallina`, and `Rainfall_Le_Croci`. These columns also **share the same values**. Therefore, we can assume that these are duplicates.

The datasets for Aquifer Doganella and Aquifer Luco share the column names `Depth_to_Groundwater_Pozzo_1`, `Depth_to_Groundwater_Pozzo_3`, `Depth_to_Groundwater_Pozzo_4`, `Volume_Pozzo_1`, `Volume_Pozzo_3` and `Volume_Pozzo_4`. These columns **do not share the same values**. Therefore, we can assume that these are not duplicates.


In [ ]:
print("River Arno and Lake Biliancino share the same values for column Rainfall_S_Piero: ", 
      river_arno_df[river_arno_df.Date.isin(lake_biliancino_df.Date.unique()) & river_arno_df.Rainfall_S_Piero.notna()]['Rainfall_S_Piero'].reset_index(drop=True)\
        .equals(lake_biliancino_df[lake_biliancino_df.Rainfall_S_Piero.notna()]['Rainfall_S_Piero'].reset_index(drop=True)))

print("River Arno and Lake Biliancino share the same values for column Rainfall_Mangona: ", 
      river_arno_df[river_arno_df.Date.isin(lake_biliancino_df.Date.unique()) & river_arno_df.Rainfall_Mangona.notna()]['Rainfall_Mangona'].reset_index(drop=True)\
        .equals(lake_biliancino_df[lake_biliancino_df.Rainfall_Mangona.notna()]['Rainfall_Mangona'].reset_index(drop=True)))

print("River Arno and Lake Biliancino share the same values for column Rainfall_S_Agata: ", 
      river_arno_df[river_arno_df.Date.isin(lake_biliancino_df.Date.unique()) & river_arno_df.Rainfall_S_Agata.notna()]['Rainfall_S_Agata'].reset_index(drop=True)\
        .equals(lake_biliancino_df[lake_biliancino_df.Rainfall_S_Agata.notna()]['Rainfall_S_Agata'].reset_index(drop=True)))

print("River Arno and Lake Biliancino share the same values for column Rainfall_Cavallina: ", 
      river_arno_df[river_arno_df.Date.isin(lake_biliancino_df.Date.unique()) & river_arno_df.Rainfall_Cavallina.notna()]['Rainfall_Cavallina'].reset_index(drop=True)\
        .equals(lake_biliancino_df[lake_biliancino_df.Rainfall_Cavallina.notna()]['Rainfall_Cavallina'].reset_index(drop=True)))

print("River Arno and Lake Biliancino share the same values for column Rainfall_Le_Croci: ", 
      river_arno_df[river_arno_df.Date.isin(lake_biliancino_df.Date.unique()) & river_arno_df.Rainfall_Le_Croci.notna()]['Rainfall_Le_Croci'].reset_index(drop=True)\
        .equals(lake_biliancino_df[lake_biliancino_df.Rainfall_Le_Croci.notna()]['Rainfall_Le_Croci'].reset_index(drop=True)))

print("Aquifer Doganella and Aquifer Luco share the same values for column Depth_to_Groundwater_Pozzo_1: ", 
      aquifer_doganella_df[aquifer_doganella_df.Date.isin(aquifer_luco_df.Date.unique()) & aquifer_doganella_df.Depth_to_Groundwater_Pozzo_1.notna()]['Depth_to_Groundwater_Pozzo_1'].reset_index(drop=True)\
        .equals(aquifer_luco_df[aquifer_luco_df.Depth_to_Groundwater_Pozzo_1.notna()]['Depth_to_Groundwater_Pozzo_1'].reset_index(drop=True)))

print("Aquifer Doganella and Aquifer Luco share the same values for column Depth_to_Groundwater_Pozzo_3: ", 
      aquifer_doganella_df[aquifer_doganella_df.Date.isin(aquifer_luco_df.Date.unique()) & aquifer_doganella_df.Depth_to_Groundwater_Pozzo_3.notna()]['Depth_to_Groundwater_Pozzo_3'].reset_index(drop=True)\
        .equals(aquifer_luco_df[aquifer_luco_df.Depth_to_Groundwater_Pozzo_3.notna()]['Depth_to_Groundwater_Pozzo_3'].reset_index(drop=True)))

print("Aquifer Doganella and Aquifer Luco share the same values for column Depth_to_Groundwater_Pozzo_4: ", 
      aquifer_doganella_df[aquifer_doganella_df.Date.isin(aquifer_luco_df.Date.unique()) & aquifer_doganella_df.Depth_to_Groundwater_Pozzo_4.notna()]['Depth_to_Groundwater_Pozzo_4'].reset_index(drop=True)\
        .equals(aquifer_luco_df[aquifer_luco_df.Depth_to_Groundwater_Pozzo_4.notna()]['Depth_to_Groundwater_Pozzo_4'].reset_index(drop=True)))

print("Aquifer Doganella and Aquifer Luco share the same values for column Volume_Pozzo_1: ", 
      aquifer_doganella_df[aquifer_doganella_df.Date.isin(aquifer_luco_df.Date.unique()) & aquifer_doganella_df.Volume_Pozzo_1.notna()]['Volume_Pozzo_1'].reset_index(drop=True)\
        .equals(aquifer_luco_df[aquifer_luco_df.Volume_Pozzo_1.notna()]['Volume_Pozzo_1'].reset_index(drop=True)))

print("Aquifer Doganella and Aquifer Luco share the same values for column Volume_Pozzo_3: ", 
      aquifer_doganella_df[aquifer_doganella_df.Date.isin(aquifer_luco_df.Date.unique()) & aquifer_doganella_df.Volume_Pozzo_3.notna()]['Volume_Pozzo_3'].reset_index(drop=True)\
        .equals(aquifer_luco_df[aquifer_luco_df.Volume_Pozzo_3.notna()]['Volume_Pozzo_3'].reset_index(drop=True)))

print("Aquifer Doganella and Aquifer Luco share the same values for column Volume_Pozzo_4: ", 
      aquifer_doganella_df[aquifer_doganella_df.Date.isin(aquifer_luco_df.Date.unique()) & aquifer_doganella_df.Volume_Pozzo_4.notna()]['Volume_Pozzo_4'].reset_index(drop=True)\
        .equals(aquifer_luco_df[aquifer_luco_df.Volume_Pozzo_4.notna()]['Volume_Pozzo_4'].reset_index(drop=True)))

In [ ]:
waterbodies_df = aquifer_auser_df.merge(aquifer_doganella_df, on='Date', how='outer')
waterbodies_df = waterbodies_df.merge(aquifer_luco_df, on='Date', how='outer')
waterbodies_df = waterbodies_df.merge(aquifer_petrignano_df, on='Date', how='outer')
waterbodies_df = waterbodies_df.merge(lake_biliancino_df[['Date','Temperature_Le_Croci','Lake_Level', 'Flow_Rate']], on='Date', how='outer') # Only merge specific columns because 'Rainfall_S_Piero', 'Rainfall_Mangona', 'Rainfall_S_Agata', 'Rainfall_Cavallina', 'Rainfall_Le_Croci' are shared with river_arno_df
waterbodies_df = waterbodies_df.merge(river_arno_df, on='Date', how='outer')
waterbodies_df = waterbodies_df.merge(water_spring_amiata_df, on='Date', how='outer')
waterbodies_df = waterbodies_df.merge(water_spring_lupa_df, on='Date', how='outer')
waterbodies_df = waterbodies_df.merge(water_spring_madonna_df, on='Date', how='outer')

waterbodies_df['Date_dt'] = pd.to_datetime(waterbodies_df.Date, format = '%d/%m/%Y')
waterbodies_df = waterbodies_df.sort_values(by='Date_dt').reset_index(drop=True)
waterbodies_df

# Exploratory Data Analysis

In this section, we are going to explore the datasets.

## Geographical Analysis
Next, let's plot the locations for which we have rainfall and temperature data to get a feeling for the data. For this, we will **retrieve the coordinates** for each location. Next, we will **cluster the locations by KMeans** according to their coordinates and plot them on a map.

In [ ]:
geolocator = Nominatim(user_agent='myapplication')

city_locations_df = pd.DataFrame(columns=['city', 'lat', 'lon'] )
for city in waterbodies_df.columns[waterbodies_df.columns.str.startswith('Temperature')]:
    city = city.split('Temperature_')[1]
        
    city_dict = {}
    city_dict['city'] = city

    city = re.sub('_', ' ', city)
    city = re.sub('S ', 'San ', city)
    # Manunal corrections of city names for geolocator
    if city == 'San Agata':
        city = "Sant’Agata"
    try:
        location = geolocator.geocode(f"{city}, Italy")
        #folium.Marker([location.raw['lat'],location.raw['lon']], popup=city, icon=folium.Icon(color='blue')).add_to(m)
        city_dict['lat'] = location.raw['lat']
        city_dict['lon'] = location.raw['lon']
    except:
        # Manual coordinates for missing cities:
        if city == 'Monteroni Arbia Biena':
            city_dict['lat'] = 43.228279
            city_dict['lon'] = 11.4021433
    
    # Manual coordinates for wrong locations:
    if city == 'Monte Serra':
        city_dict['lat'] = 43.750833
        city_dict['lon'] = 10.555278
    elif city == 'Laghetto Verde':
        city_dict['lat'] = 44.5104966
        city_dict['lon'] = 10.4032786
    city_locations_df = city_locations_df.append(city_dict, ignore_index=True)

for city in waterbodies_df.columns[waterbodies_df.columns.str.startswith('Rainfall')]:
    city = city.split('Rainfall_')[1]

    city_dict = {}
    city_dict['city'] = city
    
    city = re.sub('_', ' ', city)
    city = re.sub('S ', 'San ', city)
    # Manunal corrections of city names for geolocator
    if city == 'San Agata':
        city = "Sant’Agata"
    try:
        location = geolocator.geocode(f"{city}, Italy")
        #folium.Marker([location.raw['lat'],location.raw['lon']], popup=city, icon=folium.Icon(color='blue')).add_to(m)
        city_dict['lat'] = location.raw['lat']
        city_dict['lon'] = location.raw['lon']
    except:
        # print(f'Could not find coordinates for {city}')
        # Manual coordinates for missing cities:
        if city == 'Monticiano la Pineta':
            city_dict['lat'] = 43.1335066
            city_dict['lon'] = 11.2408464
        elif city == 'Ponte Orgia':
            city_dict['lat'] = 43.2074581
            city_dict['lon'] = 11.2504416
        elif city == 'Monteroni Arbia Biena':
            city_dict['lat'] = 43.228279
            city_dict['lon'] = 11.4021433
            
    # Manual coordinates for wrong locations:
    if city == 'Monte Serra':
        city_dict['lat'] = 43.750833
        city_dict['lon'] = 10.555278
    elif city == 'Laghetto Verde':
        city_dict['lat'] = 44.5104966
        city_dict['lon'] = 10.4032786
            
    city_locations_df = city_locations_df.append(city_dict, ignore_index=True)

# Cities with both temperature and rainfall data
#temp = city_locations_df.city.value_counts().to_frame()
#print(list(temp[temp.city==2].index.values))

city_locations_df = city_locations_df.sort_values(by='city').drop_duplicates().reset_index(drop=True)
city_locations_df.lat = city_locations_df.lat.astype(float)
city_locations_df.lon = city_locations_df.lon.astype(float)

# Cluster locations
N_CLUSTERS = 18
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42).fit(city_locations_df[['lat', 'lon']]) #1
city_locations_df['cluster'] = kmeans.labels_

# Re-arrange clusters by lat/lon
city_locations_df['cluster_center_lat'] = city_locations_df['cluster'].apply(lambda x: kmeans.cluster_centers_[x,0])
city_locations_df['cluster_center_lon'] = city_locations_df['cluster'].apply(lambda x:kmeans.cluster_centers_[x, 1])
city_locations_df = city_locations_df.sort_values(by='cluster').reset_index(drop=True)
city_locations_df = city_locations_df.sort_values(by=['cluster_center_lat','cluster_center_lon']).reset_index(drop=True)

# Rename clusters in order 
cluster_dict = {}
for old, new in enumerate(city_locations_df.cluster.unique()):
    cluster_dict[new] = old

city_locations_df.cluster = city_locations_df.cluster.replace(cluster_dict)

# Plot on map
m = folium.Map(location=[41.8719, 12.5674], tiles='cartodbpositron',zoom_start=5)

colors = ['beige', 'orange','pink', 'lightred','red',  'darkred','purple','darkpurple', 
          'lightblue', 'cadetblue', 'blue', 'darkblue',
         'lightgreen', 'green', 'darkgreen', 'black', 'gray','lightgray', 'white']


geolocator = Nominatim(user_agent='myapplication')
for i in city_locations_df.index:
    folium.Marker([city_locations_df.iloc[i].lat, 
                  city_locations_df.iloc[i].lon],
                  popup=city_locations_df.iloc[i].city, 
                  icon=folium.Icon(color=colors[city_locations_df.iloc[i].cluster])).add_to(m)

m

Next, we will calculate the **distances between the locations with the haversine formula** and verify the clusters.

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Function copied from https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """

    R = 6372.8 # Earth radius in kilometers

    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))

    return round(R * c, 0)

distances = np.zeros((len(city_locations_df.index), len(city_locations_df.index)))
for i in (city_locations_df.index):
    for j in city_locations_df.index:
        distances[i, j] = haversine(city_locations_df.iloc[i].lon, 
                                    city_locations_df.iloc[i].lat, 
                                    city_locations_df.iloc[j].lon, 
                                    city_locations_df.iloc[j].lat)

distances_df = pd.DataFrame(columns = city_locations_df.city.values, index= city_locations_df.city.values, data=distances)



f, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 12))
ax.set_title('Clustered Distances between Locations [km]', fontsize=16)
sns.heatmap(distances_df, cmap='Blues')
ax.add_patch(Rectangle((0, 0), 1, 1, fill=False, alpha=1, color='Black', lw=2)) # Cluster 0
ax.add_patch(Rectangle((1, 1), 1, 1, fill=False, alpha=1, color='Black', lw=2)) # Cluster 1
ax.add_patch(Rectangle((2, 2), 1, 1, fill=False, alpha=1, color='Black', lw=2)) # Cluster 2
ax.add_patch(Rectangle((3, 3), 2, 2, fill=False, alpha=1, color='Black', lw=2)) # Cluster 3
ax.add_patch(Rectangle((5, 5), 1, 1, fill=False, alpha=1, color='Black', lw=2)) # Cluster 4
ax.add_patch(Rectangle((6, 6), 3, 3, fill=False, alpha=1, color='Black', lw=2)) # Cluster 5
ax.add_patch(Rectangle((9, 9), 1, 1, fill=False, alpha=1, color='Black', lw=2)) # Cluster 6
ax.add_patch(Rectangle((10, 10), 1, 1, fill=False, alpha=1, color='Black', lw=2)) # Cluster 7
ax.add_patch(Rectangle((11, 11), 10, 10, fill=False, alpha=1, color='Black', lw=2)) # Cluster 8
ax.add_patch(Rectangle((21, 21), 1, 1, fill=False, alpha=1, color='Black', lw=2)) # Cluster 9
ax.add_patch(Rectangle((22, 22), 3, 3, fill=False, alpha=1, color='Black', lw=2)) # Cluster 10
ax.add_patch(Rectangle((25, 25), 1, 1, fill=False, alpha=1, color='Black', lw=2)) # Cluster 11
ax.add_patch(Rectangle((26, 26), 4, 4, fill=False, alpha=1, color='Black', lw=2)) # Cluster 12
ax.add_patch(Rectangle((30, 30), 5, 5, fill=False, alpha=1, color='Black', lw=2)) # Cluster 13
ax.add_patch(Rectangle((35, 35), 4, 4, fill=False, alpha=1, color='Black', lw=2)) # Cluster 14
ax.add_patch(Rectangle((39, 39), 7, 7, fill=False, alpha=1, color='Black', lw=2)) # Cluster 15
ax.add_patch(Rectangle((46, 46), 1, 1, fill=False, alpha=1, color='Black', lw=2)) # Cluster 16
ax.add_patch(Rectangle((47, 47), 1, 1, fill=False, alpha=1, color='Black', lw=2)) # Cluster 17

plt.show()



## Seasonality

The challenge description gives us a hint about seasonality. Therefore, we will analyse the seasonality of the features in this section. For this purpose, we will **create time features: `year`, `month`, `day_in_year`, `week_in_year`.** 

> During fall and winter waterbodies are refilled, but during spring and summer they start to drain.


Before, we look at the seasonality, let's get a feeling about how many years of data we are looking at for each dataset. For the datasets for aquifier Auser, aquifier Luco, river Arno, and water spring Amiata, we have **more than 20 years of data.** On the other side, water spring Madonna only contains information about the past 8 years.

In [ ]:
def get_time_features(df):
    df['year'] = df.Date.apply(lambda x: x.split('/')[2] if x==x else x)
    df['month'] = df.Date.apply(lambda x: x.split('/')[1]if x==x else x)
    df['day_in_year'] = df.Date.apply(lambda x: datetime(int(x.split('/')[2]), int(x.split('/')[1]), int(x.split('/')[0]), 0, 0).timetuple().tm_yday if x==x else x)
    df['week_in_year'] = df.day_in_year.apply(lambda x: int(x/7) if x==x else x)
    return df

aquifer_auser_df = get_time_features(aquifer_auser_df)
aquifer_doganella_df = get_time_features(aquifer_doganella_df)
aquifer_luco_df = get_time_features(aquifer_luco_df)
aquifer_petrignano_df = get_time_features(aquifer_petrignano_df)
lake_biliancino_df = get_time_features(lake_biliancino_df)
river_arno_df = get_time_features(river_arno_df)
water_spring_amiata_df = get_time_features(water_spring_amiata_df)
water_spring_lupa_df = get_time_features(water_spring_lupa_df)
water_spring_madonna_df = get_time_features(water_spring_madonna_df)


temp_df = (pd.DataFrame({'year' : aquifer_auser_df.year.unique(), 'dataset': 'Aquifier Auser'}))
temp_df = temp_df.append(pd.DataFrame({'year' : aquifer_doganella_df.year.unique(), 'dataset': 'Aquifier Doganella'}))
temp_df = temp_df.append(pd.DataFrame({'year' : aquifer_luco_df.year.unique(), 'dataset': 'Aquifier Luco'}))
temp_df = temp_df.append(pd.DataFrame({'year' : aquifer_petrignano_df.year.unique(), 'dataset': 'Aquifier Petrignano'}))
temp_df = temp_df.append(pd.DataFrame({'year' : lake_biliancino_df.year.unique(), 'dataset': 'Lake Biliancino'}))
temp_df = temp_df.append(pd.DataFrame({'year' : river_arno_df.year.unique(), 'dataset': 'River Arno'}))
temp_df = temp_df.append(pd.DataFrame({'year' : water_spring_amiata_df.year.unique(), 'dataset': 'Water Spring Amiata'}))
temp_df = temp_df.append(pd.DataFrame({'year' : water_spring_lupa_df.year.unique(), 'dataset': 'Water Spring Lupa'}))
temp_df = temp_df.append(pd.DataFrame({'year' : water_spring_madonna_df.year.unique(), 'dataset': 'Water Spring  Madonna'}))

temp_df['dummy'] = 1
temp_df = temp_df[temp_df.year.notna()]
temp_df = temp_df.pivot(index='dataset', columns='year')['dummy']

f, ax = plt.subplots(nrows=1, ncols=1, figsize=(20,6))

sns.heatmap(temp_df, cmap='Blues', linewidth=1, ax=ax, vmin=0, vmax=1)


ax.set_ylabel('Dataset', fontsize=16)
ax.set_xlabel('Year', fontsize=16)
ax.set_title('Yearly Data Availability for each Dataset', fontsize=16)
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(14) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(14) 
plt.show()

# Missing Data
We can see that there is a lot of missing data in the merged dataset. For simplification reasons, we will drop all data before 2000 since there are only 5 features available. Losing the data is 1998 and 1999 is not evaluated as cricital since there is still 20 years of data left.

In [ ]:
waterbodies_df['year'] = waterbodies_df.Date.apply(lambda x: x.split('/')[2] if x==x else x)
waterbodies_df['year'] = waterbodies_df['year'].astype(int)

temp = waterbodies_df[[c for c in waterbodies_df.columns if c != 'year']].isna().astype(int)
temp['year'] = waterbodies_df.year
temp = temp.groupby('year').mean()

f, ax = plt.subplots(nrows=1, ncols=1, figsize=(30,6))

sns.heatmap(temp,  
            cmap='binary', vmin=-0.1, vmax=1, ax=ax, linewidth=1)#[0])
ax.set_ylabel('Year', fontsize=16)
ax.set_xlabel('Feature', fontsize=16)
ax.set_title('Missing Values', fontsize=16)

plt.show()

waterbodies_df = waterbodies_df[waterbodies_df.year>= 2000].reset_index(drop=True)

## Missing Values and Implausible Zero Values
From below plot, we can see that the temperature and the rainfall columns have a lot of missing values (NaN). Also, we can see that there are a lot **implausible temperature values of 0°C**. For the rainfall data, the zero values seem plausible.

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=2, figsize=(24,6))

sns.heatmap(waterbodies_df[waterbodies_df.columns[waterbodies_df.columns.str.startswith('Rainfall')]].isna().astype(int),  
            cmap='binary', linewidth=0, vmin=0, vmax=1, ax=ax[0])
ax[0].set_ylabel('Row', fontsize=16)
ax[0].set_xlabel('Column', fontsize=16)
ax[0].set_title('Missing Rainfall Values', fontsize=16)

sns.heatmap(waterbodies_df[waterbodies_df.columns[waterbodies_df.columns.str.startswith('Temperature')]].isna().astype(int),  
            cmap='binary', linewidth=0, vmin=0, vmax=1, ax=ax[1])
ax[1].set_ylabel('Row', fontsize=16)
ax[1].set_xlabel('Year', fontsize=16)
ax[1].set_title('Missing Temperature Values', fontsize=16)

plt.show()

f, ax = plt.subplots(nrows=1, ncols=2, figsize=(24,6))

sns.heatmap((waterbodies_df[waterbodies_df.columns[waterbodies_df.columns.str.startswith('Rainfall')]] == 0).astype(int),  
            cmap='binary', linewidth=0, vmin=0, vmax=1, ax=ax[0])
ax[0].set_ylabel('Row', fontsize=16)
ax[0].set_xlabel('Column', fontsize=16)
ax[0].set_title('Zero Rainfall Values', fontsize=16)

sns.heatmap((waterbodies_df[waterbodies_df.columns[waterbodies_df.columns.str.startswith('Temperature')]] == 0).astype(int),  
            cmap='binary', linewidth=0, vmin=0, vmax=1, ax=ax[1])
ax[1].set_ylabel('Row', fontsize=16)
ax[1].set_xlabel('Year', fontsize=16)
ax[1].set_title('Zero Temperature Values', fontsize=16)

plt.show()

Let's have a look at the **seasonality of rainfall and temperature** first. Below plots show the overall mean rainfall and temperature over the weeks in a year. We can see that on average, the **rainfall is low in the summer months and higher in the winter months**. There is a **peak of rainfall around November**. The temperature has an almost opposite seasonality. The **temperature reaches a peak in summer**.

In [ ]:
temperature_cols = waterbodies_df.columns[waterbodies_df.columns.str.startswith('Temperature')]
waterbodies_df[temperature_cols] = waterbodies_df[temperature_cols].replace({0 : np.nan})

waterbodies_df = get_time_features(waterbodies_df)
waterbodies_df.month = waterbodies_df.month.astype(int)

columns = waterbodies_df.columns[waterbodies_df.columns.str.startswith('Rainfall') | waterbodies_df.columns.str.startswith('Temperature')]

temp = waterbodies_df.groupby('week_in_year')[columns].mean().reset_index(drop=False)

f, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
f.suptitle('Seasonality of Rainfall and Temperature', fontsize=16)

evenly_spaced_interval = np.linspace(0, 1, len(temp.columns[temp.columns.str.startswith('Rainfall')])+1)
colors = [cm.Spectral(x) for x in evenly_spaced_interval]
for i, col in enumerate(temp.columns[temp.columns.str.startswith('Rainfall')]):
    sns.lineplot(data=temp, x='week_in_year', y=col, ax=ax[0], label=col, color=colors[i+1])
ax[0].legend(bbox_to_anchor=(0.6, -0.1, 0, 0))
ax[0].set_ylabel('Rainfall', fontsize=14)
ax[0].set_xlabel('Week in Year', fontsize=14)
ax[0].set_ylim([0, 20])

evenly_spaced_interval = np.linspace(0, 1, len(temp.columns[temp.columns.str.startswith('Temperature')])+1)
colors = [cm.Spectral(x) for x in evenly_spaced_interval]
for i, col in enumerate(temp.columns[temp.columns.str.startswith('Temperature')]):
    sns.lineplot(data=temp, x='week_in_year', y=col, ax=ax[1], label=col, color=colors[i+1])
ax[1].legend(bbox_to_anchor=(0.6, -0.1, 0, 0))
ax[1].set_ylabel('Temperature [°C]', fontsize=14)
ax[1].set_xlabel('Week in Year', fontsize=14)  
ax[1].set_ylim([0, 30])

plt.show()

For Velletri, Settefrati, Pentolina, Monteroni Arbia Biena, Laghetto Verde, Le Croci, Abbadia San Salvatore, San Fiora, Bastia Umbra, Mensano, Orentano, Monte Serra, Monteporzio, and Siena Poggio al Vento, we have **both rainfall and temperature data**.
The mean values of **Siena Poggio al Vento, and Mensano seem implausible** since they are lower than the temperatures in other Italian location. This could however be caused by the implausible zero values as seen in above section.
Water spring Lupa, could be the most challenging to predict since it only has two feature (`Date` and `Rainfall_Terni`).

### Imputing Temperature Data by Prediction
As a preprocessing step of the data, we will fill the NaN values and replace the implausible values as well.
First, the implausible values of 0°C will be replaced with NaN values. Then all NaN values will be filled by prediction. For this, we will build a small prediction model.

We will be using LightGBM with a 5 fold CV. The features are the temperature columns as well as the date features.
Although, the mean squared error is quite high for the below shown example, we can see that the rolling mean for the predicted values is very close to the rolling mean of the ground truth. Although the models performance is not very good, we can evaluate it as sufficient for filling the missing values.

(Work in progress: Maybe utilize above clusters as a second model to improve prediction)

In [ ]:
import datetime

params = {'num_leaves': 24,
          'objective': 'regression',
          'max_depth': 16,
          'learning_rate': 0.005,
          "metric": 'rmse',
          "verbosity": -1,
          'random_state': 42,
          'verbose': -1,
         }

NFOLDS = 5

def predict_missing_temperature(target, debug=False):

    if debug:
        f, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 6))
        ax.set_title(f'Temperatures for {target}', fontsize=16)

        sns.lineplot(x=waterbodies_df.Date_dt, y=waterbodies_df[target], label=target, )
        sns.scatterplot(x=waterbodies_df.Date_dt, y=waterbodies_df[target].isna().apply(lambda x: 15 if x else np.nan), color='red', linewidth=0, label='To predict' )
        ax.set_xlim([datetime.date(2000, 1, 1), datetime.date(2020, 6, 30)])
        plt.show()

    test_df = waterbodies_df.copy()
    train_df = waterbodies_df[waterbodies_df[target].notna()]

    features = [c for c in (list(temperature_cols) + list(['month', 'day_in_year', 'week_in_year'])) if c != target]

    X = train_df[features]
    y = train_df[[target]]
    X_test = test_df[features]

    folds = KFold(n_splits=NFOLDS)
    splits = folds.split(X, y)
    y_preds = np.zeros(X_test.shape[0])
    y_oof = np.zeros(X.shape[0])
    score = 0

    for fold_n, (train_index, valid_index) in enumerate(splits):
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        dtrain = lgb.Dataset(X_train, y_train, params= {'verbose': -1})
        dvalid = lgb.Dataset(X_valid, y_valid, params= {'verbose': -1})

        # For analysis set 'verbose_eval' to 200
        clf = lgb.train(params, dtrain, 10000, valid_sets = [dtrain, dvalid], verbose_eval=False,  early_stopping_rounds=300)

        y_pred_valid = clf.predict(X_valid)
        y_oof[valid_index] = y_pred_valid
        #print(f"Fold {fold_n + 1} | RSME: {mean_squared_error(y_valid.values.reshape(-1), y_pred_valid)}")
        y_preds += clf.predict(X_test) / NFOLDS

        del X_train, X_valid, y_train, y_valid
        gc.collect()

    #print(f"Predicting target {target} with RSME: {mean_squared_error(y, y_oof)}")
    if debug:
        f, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 4))
        ax.set_title(f'Rolling Mean Temperatures for {target} \n(Rolling Window: 28 Days, RSME: {mean_squared_error(y, y_oof)})', fontsize=16)
        sns.lineplot(x=waterbodies_df.Date_dt, y=pd.Series(y_preds).rolling(28).mean(), label='Predicted', color='tomato')
        sns.lineplot(x=waterbodies_df.Date_dt, y=waterbodies_df[target].rolling(28).mean(), label='Ground Truth', color='dodgerblue')
        ax.set_xlabel('Date', fontsize=14)
        ax.set_xlim([datetime.date(2000, 1, 1), datetime.date(2020, 6, 30)])
        plt.show()
    return pd.Series(y_preds)

for target in temperature_cols:
    waterbodies_df[target] = np.where(waterbodies_df[target].isna(), predict_missing_temperature(target, True), waterbodies_df[target])

Plotting the rolling mean temperature values shows that in 2014 the temperature values for `Temperature_Siena_Poggio_al_Vento`, `Temperature_Mensano`, and `Temperature_Pentolina` is irregular incomparison to other locations and in comparison to the other years.

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 6))
evenly_spaced_interval = np.linspace(0, 1, len(temperature_cols))#+1)
colors = [cm.Spectral(x) for x in evenly_spaced_interval]
for i, target in enumerate(temperature_cols):
    sns.lineplot(x=waterbodies_df.Date_dt, y=waterbodies_df[target].rolling(28).mean(), label=target, color=colors[i])#+1])
    ax.set_xlim([datetime.date(2000, 1, 1), datetime.date(2020, 6, 30)])

plt.legend(bbox_to_anchor = (1.2,0.5), loc = "center")
plt.show()

# Work in Progress...

In [ ]:
'''
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 4))
ax.set_title(f'Rolling Mean Temperatures for {target} \n(Rolling Window: 28 Days, RSME: {mean_squared_error(y, y_oof)})', fontsize=16)
sns.lineplot(x=waterbodies_df.Date_dt, y=waterbodies_df.Lake_Level, label='Ground Truth', color='dodgerblue')
ax.set_xlabel('Date', fontsize=14)
ax.set_xlim([datetime.date(1998, 1, 1), datetime.date(2020, 6, 30)])
plt.show()



aquifer_auser_df 
aquifer_doganella_df 
aquifer_luco_df
aquifer_petrignano_df
lake_biliancino_df
river_arno_df
water_spring_amiata_df
water_spring_lupa_df
water_spring_madonna_df

'''
